### Initial Setup

pip install Flask-PyMongo

import time

from selenium import webdriver



driver = webdriver.Chrome()  # Optional argument, if not specified will search path.

driver.get('http://www.google.com/');

time.sleep(5) # Let the user actually see something!

search_box = driver.find_element_by_name('q')

search_box.send_keys('ChromeDriver')

search_box.submit()

time.sleep(5) # Let the user actually see something!

driver.quit()

In [1]:
! which chromedriver

/c/Users/mjensen/Anaconda3/envs/PythonData/chromedriver


In [2]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News (https://redplanetscience.com/)

In [4]:
# URL for NASA news page to be scraped
news_url = 'https://redplanetscience.com/'
browser.visit( news_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs( html, 'html.parser')

In [5]:
# Retrieve the latest news title
title = soup.find_all( 'div', class_ = 'content_title')[0].text

# Retrieve the latest news paragraph
paragraph = soup.find_all( 'div', class_ = 'article_teaser_body')[0].text
print( title)
print( '-------------------')
print( paragraph)

My Culture, My Voice
-------------------
In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.


### JPL Mars Space Images - Featured Image (https://spaceimages-mars.com/)

In [6]:
# URL for JPL Featured Image
space_images_url = 'https://spaceimages-mars.com/'
browser.visit( space_images_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs( html, 'html.parser')

In [7]:
# Find featured image and print source url
featured_image = soup.find_all( 'img', class_ = 'headerimage fade-in')[0]['src']
featured_image_url = space_images_url + featured_image
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


### Mars Facts (https://galaxyfacts-mars.com/)

In [8]:
#URL for Mars Facts Website
facts_url = 'https://galaxyfacts-mars.com/'

# Use pandas to read table of facts comparing Mars and Earth
facts_table = pd.read_html(facts_url, header=0)
facts_table[0]

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
# Use pandas to read table of Mars facts
facts_table = pd.read_html(facts_url)
facts_df = facts_table[1]
facts_df.columns =['Description', 'Value']
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
# Convert to HTML
facts_html = facts_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
  

### Mars Hemispheres (https://marshemispheres.com/)

In [11]:
# URL for Mars Hemispheres Images
hemispheres_url = 'https://marshemispheres.com/'
browser.visit( hemispheres_url)

# Create HTML object
html = browser.html

# Parse HTML with BeautifulSoup
soup = bs( html, 'html.parser')

In [12]:
items = soup.find_all('div', class_='item')

mars_hemi_urls = []
mars_hemi_titles = []

for item in items:
    mars_hemi_urls.append( hemispheres_url + item.find('a')['href'])
    mars_hemi_titles.append( item.find('h3').text.strip())

print( mars_hemi_urls)
mars_hemi_titles

['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [13]:
hemi_img_urls = []

for url in mars_hemi_urls:
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Find image urls and append to list
    source_url = hemispheres_url + soup.find('img',class_='wide-image')['src']
    hemi_img_urls.append( source_url)
    
hemi_img_urls

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [14]:
hemi_img_list = []

for i in range( len( mars_hemi_titles)):
    hemi_img_list.append({ 'title':mars_hemi_titles[i], 'img_url':hemi_img_urls[i]})

hemi_img_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [15]:
mars_dict = {
    "news_title":title,
    "news_p":paragraph,
    "featured_image_url":featured_image_url,
    "fact_table":facts_html,
    "hemisphere_images":hemi_img_list
}

In [16]:
mars_dict

{'news_title': 'My Culture, My Voice',
 'news_p': 'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.',
 'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n 

In [17]:
browser.quit()